In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from transformers import AutoModelForQuestionAnswering, pipeline

from bm25_utils import BM25Gensim
from pairwise_model import PairwiseModel
from text_utils import preprocess

tqdm.pandas()

In [ ]:
data_path = "./legal"
df_wiki_windows = pd.read_csv("./train/processed/lagal_2024_10_31_cleaned_v2.csv")
corpus_df = pd.read_csv(os.path.join(data_path, "corpus.csv"))
train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
test_df = pd.read_csv(os.path.join(data_path, "public_test.csv"))
bm25_model_stage1 = BM25Gensim("./train/processed/bm25_stage1/", None, None)

In [3]:
pairwise_model_stage1 = PairwiseModel("nguyenvulebinh/vi-mrc-base").half()
pairwise_model_stage1.load_state_dict(torch.load("./data/pairwise_v2.bin"))
pairwise_model_stage1.eval()

c:\Users\Admin\Desktop\work\projects\hackathon\zac2022-e2e-qa\env\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

PairwiseModel(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [4]:
class QAModel(nn.Module):

    def __init__(self, model_name, model_checkpoint=None, thr=0.1, device="cuda:0"):
        super(QAModel, self).__init__()
        model = AutoModelForQuestionAnswering.from_pretrained(model_name).half()
        if model_checkpoint:
            model.load_state_dict(torch.load(model_checkpoint))
        self.nlp = pipeline(
            "question-answering",
            model=model,
            tokenizer=model_name,
            device=int(device.split(":")[-1]),
        )

        self.thr = thr

    def forward(self, question, texts):

        curr_answers = []
        curr_scores = []

        for text in texts:
            QA_input = {"question": question, "context": text}
            res = self.nlp(QA_input)

            curr_answers.append(res["answer"])
            curr_scores.append(res["score"])

        return curr_scores, curr_answers

In [5]:
thr = 0.1
topk = 10
model_name = "nguyenvulebinh/vi-mrc-large"
qa_model = QAModel(model_name, thr=thr)


In [6]:
def get_answer_e2e(question):
    # Bm25 retrieval for top200 candidates
    query = question.lower()
    top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, topk=200)
    cids = [df_wiki_windows.cid.values[i] for i in top_n]
    texts = [corpus_df.text.values[df_wiki_windows.i.values[i]] for i in top_n]

    # Reranking with pairwise model for top100
    question = preprocess(question)
    ranking_preds = pairwise_model_stage1.stage1_ranking(question, texts)
    ranking_scores = ranking_preds * bm25_scores
    best_idxs = np.argsort(ranking_scores)[::-1][:100]
    cids = np.array(cids)[best_idxs]
    texts = np.array(texts)[best_idxs]
    ranking_scores =  np.array(ranking_scores)[best_idxs]

    # Question answering for top10
    curr_scores, curr_answers = qa_model(question, texts)
    curr_scores = np.array(curr_scores)*ranking_scores
    
    best_idxs = np.argsort(curr_scores)[::-1][:10]

    curr_scores = np.array(curr_scores)[best_idxs]
    curr_answers = np.array(curr_answers)[best_idxs]
    cids = cids[best_idxs]
    texts = texts[best_idxs]

    return curr_scores, cids, texts, curr_answers

In [31]:
id = 712
print(train_df.iloc[id].question)
print(train_df.iloc[id].context)

Việc tổ chức hội nghị cung cấp thông tin về phát triển sản phẩm trên môi trường mạng được thực hiện theo quy mô như thế nào?
['Tổ chức, tham gia các hoạt động xúc tiến thương mại trên môi trường mạng\n...\n5. Tổ chức hội nghị, hội thảo, tọa đàm, diễn đàn cung cấp thông tin về phát triển sản phẩm, ngành hàng, thị trường trên môi trường mạng\n...\nb) Quy mô: Tối thiểu 100 đơn vị Việt Nam tham gia đối với chương trình cung cấp thông tin về phát triển sản phẩm, thị trường xuất khẩu; tối thiểu 50 đơn vị nước ngoài đối với chương trình cung cấp thông tin sản phẩm, ngành hàng của Việt Nam.\n...']


In [32]:
from graph_utils import find_best_cluster
ranking_scores, cids, texts, curr_answers = get_answer_e2e(train_df.iloc[id].question)

In [33]:
best_answer = find_best_cluster(curr_answers, curr_answers[0])
index = list(curr_answers).index(best_answer)
print(best_answer)
texts[index]

Tối


'Tổ chức, tham gia các hoạt động xúc tiến thương mại trên môi trường mạng\n...\n5. Tổ chức hội nghị, hội thảo, tọa đàm, diễn đàn cung cấp thông tin về phát triển sản phẩm, ngành hàng, thị trường trên môi trường mạng\n...\nb) Quy mô: Tối thiểu 100 đơn vị Việt Nam tham gia đối với chương trình cung cấp thông tin về phát triển sản phẩm, thị trường xuất khẩu; tối thiểu 50 đơn vị nước ngoài đối với chương trình cung cấp thông tin sản phẩm, ngành hàng của Việt Nam.\n...'

In [34]:
print(texts)

['1. Tổ chức, cá nhân được khai thác, sử dụng thông tin, dữ liệu về nhà ở và thị trường bất động sản qua mạng Internet, trang điện tử không phải trả chi phí khai thác, sử dụng thông tin, dữ liệu đối với các trường hợp sau:\na) Khai thác, sử dụng danh mục thông tin, dữ liệu về nhà ở và thị trường bất động sản;\nb) Khai thác, sử dụng thông tin, dữ liệu về nhà ở và thị trường bất động sản theo quy định của pháp Luật được công khai, phổ biến rộng rãi.\n2. Đối với các thông tin, dữ liệu ngoài quy định tại khoản 1 Điều này, tổ chức, cá nhân được đăng ký cấp quyền truy cập khai thác, sử dụng qua mạng Internet, trang điện tử. Cơ quan quản lý cơ sở dữ liệu về nhà ở và thị trường bất động sản cấp quyền truy cập cho các tổ chức, cá nhân đăng ký phù hợp với đối tượng và mục đích sử dụng theo quy định.\nTrình tự đăng ký và cấp quyền khai thác, sử dụng thông tin, dữ liệu về nhà ở và thị trường bất động sản thông qua mạng internet, trang điện tử được thực hiện như sau:\na) Tổ chức, cá nhân có nhu cầu

In [35]:
for answer in curr_answers:
    print(answer)

d) Không được thay đổi,
theo thẩm quyền;
Tối
Tối
trực tiếp hoặc gửi qua đường bưu điện đến cơ quan có thẩm quyền
cấp huyện, cấp tỉnh theo quy định của Ủy ban nhân dân cấp tỉnh.
PHỤ LỤC V
quy định hiện hành và theo thực tế phát sinh được cấp có
quy định hiện hành và theo thực tế phát sinh được cấp có
tuyến,


In [15]:
results = []
for i, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    test_id = row["qid"]
    question = row["question"]
    _, cids, _, _ = get_answer_e2e(question)

    res = [test_id]
    for cid in cids:
        res.append(cid)

    results.append(" ".join([str(r) for r in res]) + "\n")

# Lưu kết quả vào tệp
with open(model_name.replace("/", "_") + "top{}.predict.txt".format(topk), "w") as f:
    f.writelines(results)

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 